In [1]:
import datetime as dt
import time
import joblib
import sys
import fxcmpy
import colorama as col
import pandas as pd
import numpy as np
from tqdm import tqdm
import talib
from scipy.signal import argrelextrema
import mplfinance as fplt
import requests
import json
import os
sys.path.append('../') 
%matplotlib inline

## PARAMETRES

In [19]:
err_allowed = 10/100
_period = 'H4'
_token = joblib.load('TOKENS/_api_token.dag') # 'bba0dc0b9f9609f72d0f775d6058ad7806487481'
_bot_token = joblib.load('TOKENS/telegram_token.dag') # '1840623335:AAGij3spTqbfSe7QBnhyGkVb-ujnKYgf2WE
_chat_id = joblib.load('TOKENS/telegram_chat_id_dev.dag') # '-519288550'  dev: '-583227893'
_server = 'demo'
_number = 10000
_lookback = 1000

## FONCTIONS

In [20]:
def telegram_message(_message):
    message = 'https://api.telegram.org/bot'+ _bot_token +  '/sendMessage?chat_id=' + _chat_id+ '&parse_mode=Markdown&text=' + _message
    send = requests.post(message)
    send.raise_for_status()


def telegram_pic(_pic):
    files = {'photo': open(_pic, 'rb')}
    message = ('https://api.telegram.org/bot'+ _bot_token + '/sendPhoto?chat_id=' + _chat_id)
    send = requests.post(message, files = files)


def tweet_it():
    import twitter
    api = twitter.Api(consumer_key='KXALdSMKE1dN4lNCYNxhWgfyU',
                        consumer_secret='xGZjEoBMKjoxxAfjsmRa4iUvxAsI3co5IyjUOpNxkhL08KZKJk',
                        access_token_key='1029626801939726337-jrbQ0vUlOGc9wI4TlmpQItPwBKrhMX',
                        access_token_secret='ea8mKXpINq2dtAmhZNAXPXQMTGLlEATmmgatQJfAOATPV')
    user = "@Go!em"
    statuses = api.GetUserTimeline(screen_name=('@Golem_FX'))
    # ecrire message
    api.PostUpdate('INVERSE CLOSE '+_ticker+' horodaté à '+str(dt.datetime.now()))


def reduce_df(df):
    df = df[['Close','CloseAsk','CloseBid','High','HighAsk','HighBid','Low','LowAsk','LowBid','Open','OpenAsk','OpenBid','Symbol','Date']]
    return(df)


def is_we(dataframe_to_check):
    IDX = dataframe_to_check.index.to_list()
    c=0
    for day in IDX:
        if day.weekday() == 5 or day.weekday() == 6:
            c += 1
    print('Nombre de samedi et dimanches présents :',c)


def load_hdd():

    df = pd.read_csv('HDD/'+_ticker+'_'+_period+'_BidAndAsk.csv')

    ##### Ajout de la colonne Symbol pour identifier le ticker
    df['Symbol'] = _ticker

    ##### On fixe la date en index sous forme de Timestamp
    df['Lindex'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
    df.set_index(pd.to_datetime(df.Lindex,format='%Y-%m-%d %H:%M:%S'),drop=True,inplace=True)

    ###### On drop les colonnes inutiles
    df = df.drop(['Date','Lindex','Time','Total Ticks'],axis=1)
    return(df,df_H1)


def drop_we(df):
    df['WE'] = np.where(((df.index.weekday == 5) | (df.index.weekday == 6)),None,df.index.weekday)
    df = df.dropna()
    df = df.drop(['WE'],axis=1)
    return(df)


def init_base():
    """[Première initialisation de la base Live à partir de la base HDD et vérifications d'usage]
    """    
    _t1 = dt.datetime.now()
    print('Début des opérations horodatée à',col.Fore.YELLOW,dt.datetime.now(),col.Style.RESET_ALL)
    
    print('\nINITIALISATION DE LA BASE\n')

    print('Ticker :',col.Fore.YELLOW,x,col.Style.RESET_ALL)

    df = load_hdd()
    df = drop_we(df)
    is_we(df)
    df = make_mid(df)
    df = reduce_df(df)

    print('Sauvegarde des Bases')
    joblib.dump(df,'BASES/'+_ticker+'_'+_period)
    print('Bases sauvegardées')
    print('\ndf :',df,'\n')
    
    _t2 = dt.datetime.now()
    print('Fin des opérations horodatée à',col.Fore.YELLOW,dt.datetime.now(),col.Style.RESET_ALL)
    print('Executé en :',(_t2 - _t1))
    return(df)


def maj(df, con,_tf,_decay=0):
    
    _fin = dt.datetime.now()
    _deb = df.index[-1]
    _debut = dt.datetime(_deb.year,_deb.month,_deb.day,_deb.hour,_deb.minute)
    # Scrap the addon & build it to be compliant with our df
    addon = get_candl(con,_debut, _fin,_tf)
    # Calculate the mid prices

    addon = make_mid(addon)
    addon = drop_we(addon)
    addon = make_mid(addon)
    addon['Symbol'] = _ticker
    addon['Date'] = addon.index
    addon['Date'] = pd.to_datetime(addon['Date'].dt.strftime(date_format='%Y-%m-%d'))

    # Concatenate the bases
    df = df.append(addon.iloc[1:,:])
    #df = df.iloc[-263570:,:]
    if _tf == 'm5':
        if (dt.datetime.now().minute - df.index[-1].minute <= 5) : # or ((dt.datetime.now().minute - df.index[-1].minute == -55)) :
            df = df.iloc[:-1,:]
            print('Cut en m5 - Dernière bougie _period récupérée :',df.index[-1])
        else :
            print('No Cut en m5 - Dernière bougie _period récupérée :',df.index[-1])

    elif _tf == 'm15':
        if (dt.datetime.now().minute - df.index[-1].minute <= 15) : # or ((dt.datetime.now().minute - df.index[-1].minute == -55)) :
            df = df.iloc[:-1,:]
            print('Cut en m15 - Dernière bougie _period récupérée :',df.index[-1])
        else :
            print('No Cut en m15 - Dernière bougie _period récupérée :',df.index[-1])

    elif _tf == 'm30':
        if (dt.datetime.now().minute - df.index[-1].minute <= 30) : # or ((dt.datetime.now().minute - df.index[-1].minute == -55)) :
            df = df.iloc[:-1,:]
            print('Cut en m30 - Dernière bougie _period récupérée :',df.index[-1])
        else :
            print('No Cut en m30 - Dernière bougie _period récupérée :',df.index[-1])
    
    if _tf == 'H1' :
        if dt.datetime.now().hour - df.index[-1].hour == _decay :
            df = df.iloc[:-1,:]
            print('Cut en H1 - Dernière bougie _period récupérée :',df.index[-1])
    
    if _tf == 'H4' :
        if dt.datetime.now().hour - df.index[-1].hour == 4 + _decay :
            df = df.iloc[:-1,:]
            print('Cut en H4 - Dernière bougie _period récupérée :',df.index[-1])
    
        else :
            print('No Cut en H4 - Dernière bougie _period récupérée :',df.index[-1])
    return(df)


def make_mid(df):
    df['Open'] = (df.OpenAsk + df.OpenBid)/2
    df['High'] = (df.HighAsk + df.HighBid)/2
    df['Low'] = (df.LowAsk + df.LowBid)/2
    df['Close'] = (df.CloseAsk + df.CloseBid)/2
    df['Symbol'] = _ticker
    df['Date'] = df.index
    df['Date'] = pd.to_datetime(df['Date'].dt.strftime(date_format='%Y-%m-%d'))
    df = drop_we(df)
    return(df)


def get_candl(con,x,_tf,_number=200):
    df = con.get_candles(x,period=_tf,number=_number).drop(['tickqty'],axis=1)
    df = df.rename(columns={'bidopen':'OpenBid','bidclose':'CloseBid','bidhigh':'HighBid','bidlow':'LowBid','askopen':'OpenAsk','askclose':'CloseAsk','askhigh':'HighAsk','asklow':'LowAsk'})
    return(df)


def conX(con,_token,_server):
    #global con
    try:
        con.is_connected()
        if con.is_connected() == True:
            print('Déjà connecté')
            print('')
        else:
            con = fxcmpy.fxcmpy(access_token=_token, log_level='error',server=_server)
            print(col.Fore.GREEN+'Connexion établie'+col.Style.RESET_ALL)
            print('Compte utilisé : ',con.get_account_ids())
            print('')
    except:
        con = fxcmpy.fxcmpy(access_token=_token, log_level='error',server=_server)
        if con.is_connected() == True:
            print(col.Fore.GREEN+'Connexion établie'+col.Style.RESET_ALL)
            print('Compte utilisé : ',con.get_account_ids())
            print('')
        else:
            print(col.Fore.RED+'Connexion non établie'+col.Style.RESET_ALL)
            print('')
    return(con)


def peak_detect(high,low,order=10):
    #print('price.shape',price.shape)
    max_idx = list(argrelextrema(high,np.greater,order=order)[0])
    #print('max_idx shape',len(max_idx))
    min_idx = list(argrelextrema(low,np.less,order=order)[0])
    #print('min_idx shape',len(min_idx))
    idx = max_idx + min_idx
    #print('idx shape',len(idx))
    idx.sort()
    CURRENT = []
    for i in idx:
        if i in max_idx:
            CURRENT.append(high[i])
        else:
            CURRENT.append(low[i])


    if idx[-1] in max_idx:
        CURRENT.append(low[len(high)-1])

    else:
        CURRENT.append(high[len(low)-1])
    
    CURRENT = CURRENT[-5:]
    _current_idx = idx[-4:] + [len(high)-1]
    #print('current_idx shape',len(current_idx))
    _start = min(_current_idx)
    _end = max(_current_idx)
    return _current_idx,CURRENT,_start,_end 


def is_gartley(moves,err_allowed):  
    XA=moves[0]
    AB=moves[1]
    BC=moves[2]
    CD=moves[3]
   
    AB_range = np.array([0.618 - err_allowed,0.618 + err_allowed])*abs(XA)
    BC_range = np.array([0.382 - err_allowed,0.886 + err_allowed])*abs(AB)
    CD_range = np.array([1.27 - err_allowed,1.618 + err_allowed])*abs(BC)
        
    if XA>0 and AB<0 and BC>0 and CD<0:
        if AB_range[0]<abs(AB)<AB_range[1] and BC_range[0]<abs(BC)<BC_range[1] and CD_range[0]<abs(CD)<CD_range[1]:
            return 1
        else:
            return np.isnan
        
    elif XA<0 and AB>0 and BC<0 and CD>0:
        if AB_range[0]<abs(AB)<AB_range[1] and BC_range[0]<abs(BC)<BC_range[1] and CD_range[0]<abs(CD)<CD_range[1]:
            return -1
        else:
            return np.isnan
    else:
        return np.isnan


def is_butterfly(moves,err_allowed):   
    XA=moves[0]
    AB=moves[1]
    BC=moves[2]
    CD=moves[3]
    
    AB_range = np.array([0.786 - err_allowed,0.786 + err_allowed])*abs(XA)
    BC_range = np.array([0.382 - err_allowed,0.886 + err_allowed])*abs(AB)
    CD_range = np.array([1.618 - err_allowed,2.618 + err_allowed])*abs(BC)
        
    if XA>0 and AB<0 and BC>0 and CD<0:
        if AB_range[0]<abs(AB)<AB_range[1] and BC_range[0]<abs(BC)<BC_range[1] and CD_range[0]<abs(CD)<CD_range[1]:
            return 1
        else:
            return np.NaN
        
    elif XA<0 and AB>0 and BC<0 and CD>0:      
        if AB_range[0]<abs(AB)<AB_range[1] and BC_range[0]<abs(BC)<BC_range[1] and CD_range[0]<abs(CD)<CD_range[1]:
            return -1
        else:
            return np.isnan
    else:
        return np.isnan       


def is_crab(moves,err_allowed):      
    XA=moves[0]
    AB=moves[1]
    BC=moves[2]
    CD=moves[3]
    
    AB_range = np.array([0.382 - err_allowed,0.618 + err_allowed])*abs(XA)
    BC_range = np.array([0.382 - err_allowed,0.886 + err_allowed])*abs(AB)
    CD_range = np.array([2.24 - err_allowed,3.618 + err_allowed])*abs(BC)
           
    if XA>0 and AB<0 and BC>0 and CD<0:
        if AB_range[0]<abs(AB)<AB_range[1] and BC_range[0]<abs(BC)<BC_range[1] and CD_range[0]<abs(CD)<CD_range[1]:
            return 1
        else:
            return np.NaN
        
    elif XA<0 and AB>0 and BC<0 and CD>0:
        if AB_range[0]<abs(AB)<AB_range[1] and BC_range[0]<abs(BC)<BC_range[1] and CD_range[0]<abs(CD)<CD_range[1]:
            return -1
        else:
            return np.isnan
    else:
        return np.isnan      


def is_bat(moves,err_allowed):  
    XA=moves[0]
    AB=moves[1]
    BC=moves[2]
    CD=moves[3]
    
    AB_range = np.array([0.382 - err_allowed,0.5 + err_allowed])*abs(XA)
    BC_range = np.array([0.382 - err_allowed,0.886 + err_allowed])*abs(AB)
    CD_range = np.array([1.618 - err_allowed,2.618 + err_allowed])*abs(BC)
    
    if XA>0 and AB<0 and BC>0 and CD<0:
        if AB_range[0]<abs(AB)<AB_range[1] and BC_range[0]<abs(BC)<BC_range[1] and CD_range[0]<abs(CD)<CD_range[1]:
            return 1
        else:
            return np.NaN
        
    elif XA<0 and AB>0 and BC<0 and CD>0:
        if AB_range[0]<abs(AB)<AB_range[1] and BC_range[0]<abs(BC)<BC_range[1] and CD_range[0]<abs(CD)<CD_range[1]:
            return -1
        else:
            return np.isnan
    else:
        return np.isnan
    

def is_abcd(moves,err_allowed):  
    AB=moves[1]
    BC=moves[2]
    CD=moves[3]
    
    BC_range = np.array([0.618 - err_allowed,0.618 + err_allowed])*abs(AB)
    CD_range = np.array([1.618 - err_allowed,1.618 + err_allowed])*abs(BC)
    CD2_range = np.array([1 - err_allowed,1 + err_allowed])*abs(AB)
    
    if AB<0 and BC>0 and CD<0 :    
        if BC_range[0]<abs(BC)<BC_range[1] and CD_range[0]<abs(CD)<CD_range[1] \
        and CD2_range[0]<abs(CD)<CD2_range[1] :
            return 1
        else:
            return np.NaN
        
    elif AB>0 and BC<0 and CD>0 :
        if BC_range[0]<abs(AB)<BC_range[1] and CD_range[0]<abs(CD)<CD_range[1]\
        and CD2_range[0]<abs(CD)<CD2_range[1]:
            return -1
        else:
            return np.isnan
    else:
        return np.isnan    


def is_shark(moves,err_allowed):      
    XA=moves[0]
    AB=moves[1]
    BC=moves[2]
    CD=moves[3]
  
    BC_range = np.array([1.13 - err_allowed,1.618 + err_allowed])*abs(AB)
    CD_range = np.array([1.618 - err_allowed,2.24 + err_allowed])*abs(BC)
    CD_range2 = np.array([0.88 - err_allowed,1.13 + err_allowed])*abs(XA)   
    
    if XA>0 and AB<0 and BC>0 and CD<0 :        
        if BC_range[0]<abs(BC)<BC_range[1] and CD_range[0]<abs(CD)<CD_range[1] and CD_range2[0]<abs(XA)<CD_range2[1]:
            return 1
        else:
            return np.NaN
        
    elif XA<0 and AB>0 and BC<0 and CD>0 :
        if  BC_range[0]<abs(BC)<BC_range[1] and CD_range[0]<abs(CD)<CD_range[1] and CD_range2[0]<abs(XA)<CD_range2[1]:
            return -1
        else:
            return np.isnan
    else:
        return np.isnan 



In [21]:
# if __name__ == "__main__":

try:
    con
except NameError:
    con = fxcmpy.fxcmpy(access_token=_token, log_level='error',server=_server)
    print(col.Fore.GREEN+'Connexion établie'+col.Style.RESET_ALL)
    print('Compte utilisé : ',con.get_account_ids())
            
else:
    con = conX(con,_token,_server)


try:   
    _last_bougie = (con.get_candles('EUR/USD',period='m5',number=1).index.hour[0])
    _now = dt.datetime.now().hour

    if _now - _last_bougie < 0:
        _decay = _now + 24 - _last_bougie

    else:
        _decay = _now - _last_bougie


    print('\nDecay = ',_decay,'\n')
    joblib.dump(_decay,'VARS/decay.dag')
    _no_access = 0
except:
    print('\n',col.Back.RED,col.Fore.BLACK,'/!\ LES DONNEES NE SONT PAS ACCESSIBLES. PAS DE MAJ NI DE LIVE /!\ ',col.Style.RESET_ALL)
    time.sleep(0.1)
    _no_access = 1


Déjà connecté


Decay =  2 



In [22]:
TICKER_LIST = con.get_instruments()
for t in ['US.BANKS','ACA.fr','AI.fr','AIR.fr','ORA.fr','MC.fr','RNO.fr','BAYN.de','BMW.de','DPW.de','DTE.de','AZN.uk','BP.uk','GSK.uk','TSCO.uk','RDSB.uk','BRKB.us','C.us','CRM.us','MA.us','PG.us','V.us','SQ.us','CRWD.us','COIN.us','WORK.us','TWTR.us','ATMX','KO.us','MCD.us','SE.us','SBUX.us','VOW.de','DAL.us','LYFT.us','UAL.us','BOWX.us','BEKE.us','CPNG.us','NET.us','RBLX.us','ENR.de','BIDU.hk','AMD.us','DADA.us','MRNA.us','NIO.us','ABNB.us','DASH.us','AMC.us','CCIV.us','GME.us','PENN.us','PLTR.us','PLUG.us','PYPL.us','SNAP.us','SNOW.us','SPCE.us','XPEV.us']:
    TICKER_LIST.remove(t)

In [26]:
%%time

BAD_TICKERS = []
SIGNAL = []
DATE = []
TICKER = []
_compteur = 0
df_all = pd.DataFrame()
# TICKER_LIST = con.get_instruments()
for x in tqdm(TICKER_LIST):
    _ticker = x.replace('/','')
    df = get_candl(con,x,_period,_number)
    if _period == 'm5':
        if (dt.datetime.now().minute - df.index[-1].minute <= 5) : # or ((dt.datetime.now().minute - df.index[-1].minute == -55)) :
            df = df.iloc[:-1,:]
            #print('\rCut en m5 - Dernière bougie _period récupérée :',df.index[-1],end='')
        else :
            #print('\rNo Cut en m5 - Dernière bougie _period récupérée :',df.index[-1],end='')
            pass

    elif _period == 'm15':
        if (dt.datetime.now().minute - df.index[-1].minute <= 15) : # or ((dt.datetime.now().minute - df.index[-1].minute == -55)) :
            df = df.iloc[:-1,:]
            #print('\rCut en m15 - Dernière bougie _period récupérée :',df.index[-1],end='')
        else :
            #print('\rNo Cut en m15 - Dernière bougie _period récupérée :',df.index[-1],end='')
            pass

    elif _period == 'm30':
        if (dt.datetime.now().minute - df.index[-1].minute <= 30) : # or ((dt.datetime.now().minute - df.index[-1].minute == -55)) :
            df = df.iloc[:-1,:]
            #print('\rCut en m30 - Dernière bougie _period récupérée :',df.index[-1],end='')
        else :
            #print('\rNo Cut en m30 - Dernière bougie _period récupérée :',df.index[-1],end='')
            pass
    
    if _period == 'H1' :
        if dt.datetime.now().hour - df.index[-1].hour == _decay :
            df = df.iloc[:-1,:]
            #print('\rCut en H1 - Dernière bougie _period récupérée :',df.index[-1],end='')
        else :
            #print('\rNo Cut en H4 - Dernière bougie _period récupérée :',df.index[-1],end='')
            pass
    
    if _period == 'H4' :
        if dt.datetime.now().hour - df.index[-1].hour <= 4 + _decay :
            df = df.iloc[:-1,:]
            #print('\rCut en H4 - Dernière bougie _period récupérée :',df.index[-1],end='')
    
        else :
            #print('\rNo Cut en H4 - Dernière bougie _period récupérée :',df.index[-1],end='')
            pass

    df = drop_we(df)
    df = make_mid(df)
    
    price = df['Close']
    high = df['High']
    low = df['Low']
    rsi = talib.RSI(price, timeperiod=14)

    for i in range(len(price)-_lookback,len(price)-1):

        # current_idx,current_pat,start,end = peak_detect(price.values[:i+1],low.values[:i+1],high.values[:i+1])
        current_idx,current_pat,start,end = peak_detect(high.values[:i+1],low.values[:i+1])

        XA = current_pat[1] - current_pat[0]
        AB = current_pat[2] - current_pat[1]
        BC = current_pat[3] - current_pat[2]
        CD = current_pat[4] - current_pat[3]

        moves=[XA,AB,BC,CD]

        gartley = is_gartley(moves,err_allowed)
        butterfly = is_butterfly(moves,err_allowed)
        crab = is_crab(moves,err_allowed)
        bat = is_bat(moves,err_allowed)
        shark = is_shark(moves,err_allowed)
        abcd = is_abcd(moves,err_allowed)

        current_rsi = [rsi.iloc[current_idx[0]],rsi.iloc[current_idx[1]],rsi.iloc[current_idx[2]],rsi.iloc[current_idx[3]],rsi.iloc[current_idx[4]]]
        current_rsi_max = max(current_rsi)
        current_rsi_min = min(current_rsi)
        delta_price = current_pat[4] - current_pat[2]
        delta_rsi = current_rsi[4] - current_rsi[2]

        harmonics = np.array([gartley,butterfly,bat,crab,shark,abcd])
        #harmonics = np.array([abcd])
        labels = ['gartley','butterly','bat','crab','shark','abcd']
        #labels = ['abcd']
        #if (np.any(harmonics==1) and delta_price<0 and delta_rsi>0) or (np.any(harmonics==-1) and delta_price>0 and delta_rsi<0):
        if np.any(harmonics==1) or np.any(harmonics==-1):
            _compteur += 1
            for j in range(0, len(harmonics)):
                #_pic = str(i)+'.png'
                #_directory = str(df.index[current_idx[-1]]).replace('-','_').replace(' ','_').replace(':','_')+'/'
                
                #if not os.path.exists('RESULTS/'+_directory):
                #    os.makedirs('RESULTS/'+_directory)
                
                if harmonics[j]==1 or harmonics[j]==-1:
                    sense ='bearish' if harmonics[j]==-1 else 'bullish'
                    label =  sense + ' '+labels[j] +' found in TimeFrame '+_period
                    if sense =='bearish' :
                        df.loc[df.index[i],'Signal'] = -1
                    elif sense =='bullish' :
                        df.loc[df.index[i],'Signal'] = 1
                    
                    #_signals = [(df.index[current_idx[0]],current_pat[0]),(df.index[current_idx[1]],current_pat[1]),(df.index[current_idx[2]],current_pat[2])\
                        #           ,(df.index[current_idx[3]],current_pat[3]),(df.index[current_idx[4]],current_pat[4])]
                    #df['Scatter'] = np.where(df.index==df.index[current_idx[-1]],current_pat[-1],np.nan)
                    #if sense == 'bullish':
                    #    _line_price = fplt.make_addplot(df.iloc[start:i+15,:].Scatter.to_list(),type='scatter',markersize=150,marker='^',color='green')
                    #if sense == 'bearish':
                    #    _line_price = fplt.make_addplot(df.iloc[start:i+15,:].Scatter.to_list(),type='scatter',markersize=150,marker='v',color='r')
                    #fplt.plot(df.iloc[start:i+15,:], type='candle',title=_ticker+' - '+label,ylabel='Price',figscale=1.1,  datetime_format='%d-%m-%Y',alines=_signals,addplot=_line_price,\
                    #    savefig='RESULTS/'+_directory+_pic)
                    
                    #telegram_message('Ticker : '+x+'\nTimeFrame : '+_period+"\nSignal's Sense : "+sense+"\nSignal's Date : "+str(df.index[current_idx[-1]])+"\nClose's Price : "+\
                        #           str(round(price[i],4))+"\nScatter's Price : "+str(round(current_pat[-1],4)))
                    
                    #telegram_pic('RESULTS/'+_directory+_pic)
                    #time.sleep(0.1)
                    #print(df.index[current_idx],df.index[i])
                
        else:
            df.loc[df.index[i],'Signal'] = 0
    df_all = pd.concat([df_all,df])
print('\nNombre de signaux :',_compteur)
    

 30%|██▉       | 48/161 [02:48<06:15,  3.33s/it]|ERROR|2021-06-09 00:37:34,406|FXCM reject req candles/1010/H4 with status 520 and msg <!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>
<meta http-equiv="refresh" content="0">

<title>api-demo.fxcm.com | 520: Web server is returning an unknown error</title>
<meta charset="UTF-8" />
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta http-equiv="X-UA-Compatible" content="IE=Edge,chrome=1" />
<meta name="robots" content="noindex, nofollow" />
<meta name="viewport" content="width=device-width,initial-scale=1" />
<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/main.css" type="text/css" media="screen,projection" />


</head>
<body>
<div

ServerError: Request returns status code 520 and message "<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>
<meta http-equiv="refresh" content="0">

<title>api-demo.fxcm.com | 520: Web server is returning an unknown error</title>
<meta charset="UTF-8" />
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta http-equiv="X-UA-Compatible" content="IE=Edge,chrome=1" />
<meta name="robots" content="noindex, nofollow" />
<meta name="viewport" content="width=device-width,initial-scale=1" />
<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/main.css" type="text/css" media="screen,projection" />


</head>
<body>
<div id="cf-wrapper">

    

    <div id="cf-error-details" class="p-0">
        

In [ ]:
df_all['Signal'] = np.where(((df_all.Signal!=0)&(df_all.Signal!=1)&(df_all.Signal!=-1)),0,df_all.Signal)

In [8]:
df_all.Signal.value_counts()

 0.0    227667
 1.0       133
-1.0       109
Name: Signal, dtype: int64

In [68]:
%%time
df_results = pd.DataFrame()
_window = 18
_target = 0.002
_sl = 0.1
for _ticker in tqdm(df_all.Symbol.unique()):
    df_temp = df_all[df_all.Symbol==_ticker].copy()
    df_temp.sort_index(inplace=True)
    
    df_temp['Window_High_Ask'] = df_temp.HighAsk.iloc[::-1].rolling(_window).max().iloc[::-1] # Limite SL Short
    df_temp['Window_High_Bid'] = df_temp.HighBid.iloc[::-1].rolling(_window).max().iloc[::-1] # Limite Target Long
    df_temp['Window_Low_Ask'] = df_temp.LowAsk.iloc[::-1].rolling(_window).min().iloc[::-1] # Limite Target Short
    df_temp['Window_Low_Bid'] = df_temp.LowBid.iloc[::-1].rolling(_window).min().iloc[::-1] # Limite SL Long
    df_temp['Window_sl_Short'] = df_temp.CloseBid + (df_temp.CloseBid * _sl) # Short pour SL
    df_temp['Window_sl_Long'] = df_temp.CloseAsk - (df_temp.CloseAsk * _sl) # Long pour SL
    df_temp['Window_tp_Short'] = df_temp.CloseBid - (df_temp.CloseBid * _target) # Short pour TP
    df_temp['Window_tp_Long'] = df_temp.CloseAsk + (df_temp.CloseAsk * _target) # Long pour TP
    df_temp['InstantNAV'] = np.where(df_temp.Signal==1,df_temp.Window_tp_Long,np.where(df_temp.Signal==-1,df_temp.Window_tp_Short,0))

    df_results = pd.concat([df_results,df_temp])


100%|██████████| 161/161 [00:03<00:00, 51.29it/s]CPU times: user 2.57 s, sys: 547 ms, total: 3.11 s
Wall time: 3.14 s



In [70]:
df_results.InstantNAV.sum()

835590.1279771001

In [83]:
from librairies import lumberjack_trading as lbjt
TIK = ['AUD','NZD','GBP','JPY','CHF','CAD','SEK','NOK','ILS','MXN','USD','EUR','.us']
RATE = [0.776,0.721,1.3912,1/105.91,1/0.892,1/1.2681,1/8.2884,1/8.4261,1/3.2385,1/20.1564,1,1/1.21,1]
df_ratefx = pd.DataFrame(index=TIK)
df_ratefx['rate'] = RATE
x = _ticker
_period = 'H4'
_nb_bougie_exit = 18
_trigger_reengage = 0
_trigger_target = 1
_trigger_invers = 0
_trigger_sl = 0
_trigger_rsi = 0
_verbose = 1
_cash_ini = 200000
_target = 0.004
_sl = 0.001
_exposure = 2
_rate = df_ratefx.loc[x[3:],'rate']
_size = 50000 # _cash_ini / df_ratefx.loc[x[:3],'rate']
_trigger_spread = 0.025
_no_access = 0

for _ticker in df_all.Symbol.unique():
    df_temp = df_all[df_all.Symbol==_ticker].copy()
    df_temp.sort_index(inplace=True)
    _year_bottom = df_temp.index[0] 
    _year_top = df_temp.index[-1]
    _,_ = lbjt.bt(df_temp,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
            _rate,x,_target,_exposure,_size,_sl,_trigger_rsi=_trigger_rsi)




_________________________________________________

__________________________________
     ___ Period 1 : =>  m5  ___
     ___ Period 2 : =>  H1  ___
__________________________________
Ca$h Ini : 200000
Date début: 2021-02-16 06:00:00
Date fin : 2021-06-08 13:00:00
Etat verbose : 1
Nombre Bougies Exit : 18
Target : 0.004
Stop Loss : 0.001
Trigger Reengage : 0
Trigger Target : 1
Trigger Inverse : 0
Trigger StopLoss : 0
Trigger RSI : 0
Exposure : 2
_bt_report :  0
Verbose :  1
Trigger Spread : 0.025
Save :  0

_________________________________________________


Début des opérations horodatée à 2021-06-08 23:45:17.747436
_trigger_rsi : 0

Chargement de la nouvelle base


 Le rate du ticker AUDJPY est à  0.009441979038806534 
Bases chargées
TETEL process effectué
 ENTERING THE BACKTEST 
100%|██████████| 467/467 [00:00<00:00, 35543.01it/s]
(No Duration)
 For ticker  AUDJPY 
 
Total Profit & Loss : $  0 . En  0   transactions.
 
Winners Number : 0 
 
Loosers number : 0 
BT's execution time

KeyboardInterrupt: 

In [82]:
df_temp.Signal.value_counts()

0.0    469
Name: Signal, dtype: int64

In [79]:
_ticker[3:],_ticker[:3], _ticker

('USD', 'EUR', 'EURUSD')

_token,_bot_token,_chat_id 'TOKENS/telegram_token.dag'. 'TOKENS/telegram_chat_id.dag'

joblib.dump('1840623335:AAGij3spTqbfSe7QBnhyGkVb-ujnKYgf2WE','TOKENS/telegram_token.dag')

https://api.telegram.org/bot1840623335:AAGij3spTqbfSe7QBnhyGkVb-ujnKYgf2WE/getUpdates